In [50]:
import copy
import glob
import os
import time
import datetime
import csv
import matplotlib.pyplot as plt
from collections import deque

import gym
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from a2c_ppo_acktr import algo, utils
from a2c_ppo_acktr.algo import gail
from a2c_ppo_acktr.arguments import get_args
from a2c_ppo_acktr.envs import make_vec_envs
from a2c_ppo_acktr.model import Policy, Bandit_Policy
from a2c_ppo_acktr.storage import RolloutStorage, SkipReplayBuffer
from evaluation import evaluate

In [66]:
path = "./experiments/BreakoutNoFrameskip-v4/b_a2c/20220103T193642.001956/trained_models/BreakoutNoFrameskip-v4.pt"

In [67]:
actor_critic = torch.load(path)[0]
env_name = "BreakoutNoFrameskip-v4"
seed = 123
num_processes = 1
eval_log_dir = './'
device = "cpu"
img_dir = './render_img/'

RuntimeError: version_ <= kMaxSupportedFileFormatVersion INTERNAL ASSERT FAILED at /opt/conda/conda-bld/pytorch_1579022030672/work/caffe2/serialize/inline_container.cc:132, please report a bug to PyTorch. Attempted to read a PyTorch file with version 3, but the maximum supported version for reading is 2. Your PyTorch installation may be too old. (init at /opt/conda/conda-bld/pytorch_1579022030672/work/caffe2/serialize/inline_container.cc:132)
frame #0: c10::Error::Error(c10::SourceLocation, std::string const&) + 0x47 (0x7fbf35df1627 in /home/jk/anaconda3/envs/atari/lib/python3.7/site-packages/torch/lib/libc10.so)
frame #1: caffe2::serialize::PyTorchStreamReader::init() + 0x1f5b (0x7fbf37c88cbb in /home/jk/anaconda3/envs/atari/lib/python3.7/site-packages/torch/lib/libtorch.so)
frame #2: caffe2::serialize::PyTorchStreamReader::PyTorchStreamReader(std::string const&) + 0x64 (0x7fbf37c89ed4 in /home/jk/anaconda3/envs/atari/lib/python3.7/site-packages/torch/lib/libtorch.so)
frame #3: <unkno

In [61]:
envs = make_vec_envs(env_name, seed, num_processes, 0.99, eval_log_dir, device, False)

In [64]:
eval_episode_rewards = []

obs = envs.reset()
eval_recurrent_hidden_states = torch.zeros(
    num_processes, actor_critic.recurrent_hidden_state_size, device=device)
eval_masks = torch.zeros(num_processes, 1, device=device)
i = 0
while len(eval_episode_rewards) < 1:
    i+=1
    with torch.no_grad():
        _, action, _, eval_recurrent_hidden_states = actor_critic.act(
            obs,
            eval_recurrent_hidden_states,
            eval_masks,
            deterministic=True)

    # Obser reward and next obs
    obs, _, done, infos = envs.step(action)
    img_save_dir = img_dir + '{}.png'.format(i)
    plt.imsave( img_save_dir , obs[0][0])

    eval_masks = torch.tensor(
        [[0.0] if done_ else [1.0] for done_ in done],
        dtype=torch.float32,
        device=device)

    for info in infos:
        if 'episode' in info.keys():
            eval_episode_rewards.append(info['episode']['r'])

envs.close()

./render_img/1.png
./render_img/2.png
./render_img/3.png
./render_img/4.png
./render_img/5.png
./render_img/6.png
./render_img/7.png
./render_img/8.png
./render_img/9.png
./render_img/10.png
./render_img/11.png
./render_img/12.png
./render_img/13.png
./render_img/14.png
./render_img/15.png
./render_img/16.png
./render_img/17.png
./render_img/18.png
./render_img/19.png
./render_img/20.png
./render_img/21.png
./render_img/22.png
./render_img/23.png
./render_img/24.png
./render_img/25.png
./render_img/26.png
./render_img/27.png
./render_img/28.png
./render_img/29.png
./render_img/30.png
./render_img/31.png
./render_img/32.png
./render_img/33.png
./render_img/34.png
./render_img/35.png
./render_img/36.png
./render_img/37.png
./render_img/38.png
./render_img/39.png
./render_img/40.png
./render_img/41.png
./render_img/42.png
./render_img/43.png
./render_img/44.png
./render_img/45.png
./render_img/46.png
./render_img/47.png
./render_img/48.png
./render_img/49.png
./render_img/50.png
./render_

In [65]:
print(" Evaluation using {} episodes: mean reward {:.5f}\n".format(
        len(eval_episode_rewards), np.mean(eval_episode_rewards)))

 Evaluation using 1 episodes: mean reward 0.00000

